# Generate random model traffic with v2 Agents in Foundry
This notebook creates and runs an agent from AI Foundry

Best way to set it up is with [uv](https://docs.astral.sh/uv/)

1. in `agents` directory run `uv sync` (if python is missing, install it using `uv python install`)
2. in VSCode press [Ctrl+Shift+P] and select `Python: Select Interpreter`, choose the one from `agents` directory: `./agents/.venv/bin/python3.xx`

## Step 1
Get projects from foundry

In [2]:
import os
from dotenv import load_dotenv
from azure.identity.aio import DefaultAzureCredential
from azure.ai.projects.aio import AIProjectClient
from src.foundry_utils import get_ai_foundry_projects

load_dotenv(override=True)

print("🚀 Initializing AI Foundry Random Traffic Generator...")
print("=" * 60)

deployments = ["gpt-4.1-mini", "gpt-5-mini", "o3-mini"]
subscription_id = os.environ.get("AZURE_AI_FOUNDRY_SUBSCRIPTION_ID")
resource_group = os.environ.get("AZURE_AI_FOUNDRY_RESOURCE_GROUP")
account_name = os.environ.get("AZURE_AI_FOUNDRY_NAME")

print("\n📋 Configuration:")
print(f"   📍 Subscription: {subscription_id[:20]}..." if subscription_id else "   ⚠️ Subscription: Not set")
print(f"   📦 Resource Group: {resource_group}" if resource_group else "   ⚠️ Resource Group: Not set")
print(f"   🏭 Account Name: {account_name}" if account_name else "   ⚠️ Account Name: Not set")
print(f"   🤖 Available Models: {', '.join(deployments)}")

# Get all project endpoints
print("\n" + "=" * 60)
print("🔍 DISCOVERING PROJECTS")
print("=" * 60)

project_endpoints = await get_ai_foundry_projects(
    subscription_id=subscription_id,
    resource_group=resource_group,
    account_name=account_name,
)

print(f"\n📊 Found {len(project_endpoints)} project endpoint(s)")

# Create AIProjectClient and agents_utils for each project
print("\n🔧 Initializing clients...")
project_clients = []
creds = DefaultAzureCredential()

for idx, endpoint in enumerate(project_endpoints):
    if endpoint:
        project_client = AIProjectClient(endpoint=endpoint, credential=creds)
        await project_client.__aenter__()
        project_clients.append(project_client)
        print(f"   ✅ Client {idx + 1}: {endpoint[:50]}...")

# Summary
print("\n" + "=" * 60)
print("✅ SETUP COMPLETE")
print("=" * 60)
print(f"   📊 Total clients created: {len(project_clients)}")
print(f"   🤖 Models to test: {len(deployments)}")
print("\n🎉 Ready to generate random traffic!")

🚀 Initializing AI Foundry Random Traffic Generator...

📋 Configuration:
   📍 Subscription: 0721e282-2773-4021-a...
   📦 Resource Group: rg-apim-basic-public
   🏭 Account Name: ai-foundry-ijordjnftsf7w
   🤖 Available Models: gpt-4.1-mini, gpt-5-mini, o3-mini

🔍 DISCOVERING PROJECTS


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7592e24e42d0>


Status: 200

--- AI Foundry Projects ---
  Name: ai-foundry-ijordjnftsf7w/project-for-keyvault, Location: westus
    Endpoint: https://ai-foundry-ijordjnftsf7w.services.ai.azure.com/api/projects/project-for-keyvault
  Name: ai-foundry-ijordjnftsf7w/ai-project-ijordjnftsf7w-1, Location: westus
    Endpoint: https://ai-foundry-ijordjnftsf7w.services.ai.azure.com/api/projects/ai-project-ijordjnftsf7w-1
  Name: ai-foundry-ijordjnftsf7w/ai-project-ijordjnftsf7w-2, Location: westus
    Endpoint: https://ai-foundry-ijordjnftsf7w.services.ai.azure.com/api/projects/ai-project-ijordjnftsf7w-2
  Name: ai-foundry-ijordjnftsf7w/ai-project-ijordjnftsf7w-3, Location: westus
    Endpoint: https://ai-foundry-ijordjnftsf7w.services.ai.azure.com/api/projects/ai-project-ijordjnftsf7w-3

Total projects found: 4
Project names: ['ai-foundry-ijordjnftsf7w/project-for-keyvault', 'ai-foundry-ijordjnftsf7w/ai-project-ijordjnftsf7w-1', 'ai-foundry-ijordjnftsf7w/ai-project-ijordjnftsf7w-2', 'ai-foundry-ijordjnftsf

## Run Tests

In [3]:
# Send random requests to random models for each project
from src.foundry_utils import get_gateway_connections
from src.agents_utils import agents_utils

import pandas as pd
import random
import time
from openai import APIStatusError, APIError
from azure.core.exceptions import HttpResponseError

print("🎲 RANDOM TRAFFIC GENERATOR")
print("=" * 60)
print("Sending randomized requests across projects and models")
print()

# Seed random with current time to ensure different values each run
random.seed(time.time())

# Configuration
min_requests_per_project = 3
max_requests_per_project = 10
prompts = [
    "What is 2 + 2?",
    "Tell me a joke",
    "What's the capital of France?",
    "Explain quantum computing in one sentence",
    "What is the meaning of life?",
    "Write a haiku about coding",
    "What color is the sky?",
    "Name three programming languages",
]

print("⚙️ Test Configuration:")
print(f"   • Min requests per project: {min_requests_per_project}")
print(f"   • Max requests per project: {max_requests_per_project}")
print(f"   • Available prompts: {len(prompts)}")
print(f"   • Available models: {', '.join(deployments)}")
print()

results = []

for idx, project_client in enumerate(project_clients):
    endpoint = project_endpoints[idx]
    num_requests = random.randint(min_requests_per_project, max_requests_per_project)
    
    print(f"\n{'─' * 60}")
    print(f"📁 PROJECT {idx + 1}/{len(project_clients)}")
    print(f"{'─' * 60}")
    print(f"   📍 Endpoint: {endpoint[:50]}..." if endpoint else "   ⚠️ Endpoint: Unknown")
    
    gateway_connections = await get_gateway_connections(project_client)
    agents_client = agents_utils(project_client)
    connection_name = (
        gateway_connections["ai_gateway_connection_dynamic"]
        or gateway_connections["ai_gateway_connection_static"]
    )
    
    print(f"   🔗 Connection: {connection_name or '❌ Not found'}")
    print(f"   📊 Requests planned: {num_requests}")
    
    if not connection_name:
        print(f"   ⚠️ Skipping - no gateway connection found")
        continue

    openai_client = project_client.get_openai_client()
    print()

    for i in range(num_requests):
        # Pick random model and prompt
        model = random.choice(deployments)
        prompt = random.choice(prompts)
        request_id = None
        result = None
        error_msg = None
        output = None

        try:
            agent = await agents_client.create_agent(
                name="temp-agent",
                model_gateway_connection=connection_name,
                deployment_name=model,
                delete_before_create=False,
            )

            # Create conversation
            conversation = await openai_client.conversations.create(
                items=[
                    {
                        "type": "message",
                        "role": "user",
                        "content": prompt,
                    }
                ],
            )

            # Get response
            response = await openai_client.responses.create(
                conversation=conversation.id,
                extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
                input="",
            )

            request_id = response._request_id
            result = "SUCCESS"
            output = response.output_text[:100] if response.output_text else "No output"

            print(f"   ✅ [{i+1}/{num_requests}] {model} → {output[:40]}...")

        except (APIStatusError, APIError, HttpResponseError) as e:
            request_id = (
                e.response.headers.get("x-request-id", "N/A")
                if hasattr(e, "response") and e.response
                else "N/A"
            )
            result = "ERROR"
            error_msg = str(e)[:100]
            output = error_msg
            print(f"   ❌ [{i+1}/{num_requests}] {model} → {error_msg[:40]}...")

        except Exception as e:
            result = "ERROR"
            error_msg = str(e)[:100]
            output = error_msg
            print(f"   ❌ [{i+1}/{num_requests}] {model} → {error_msg[:40]}...")

        results.append(
            {
                "Project": endpoint.split("/")[-1] if endpoint else f"Project_{idx}",
                "Model": model,
                "Request #": i + 1,
                "Request ID": request_id or "N/A",
                "Prompt": prompt[:30] + "..." if len(prompt) > 30 else prompt,
                "Result": result,
                "Output": output[:50] if output else "N/A",
            }
        )

# Create DataFrame and display results
df = pd.DataFrame(results)

print("\n" + "=" * 60)
print("📊 RESULTS TABLE")
print("=" * 60)
print(df.to_string(index=False))

# Save results to CSV
csv_file = "gateway_random_traffic_results.csv"
df.to_csv(csv_file, index=False)
print(f"\n💾 Results saved to: {csv_file}")

# Summary statistics
print("\n" + "=" * 60)
print("📈 SUMMARY BY PROJECT AND MODEL")
print("=" * 60)
summary = df.groupby(["Project", "Model", "Result"]).size().unstack(fill_value=0)
print(summary)

print("\n" + "=" * 60)
print("🎯 OVERALL RESULTS")
print("=" * 60)
total = len(results)
successes = len([r for r in results if r["Result"] == "SUCCESS"])
errors = len([r for r in results if r["Result"] == "ERROR"])
success_rate = (successes / total * 100) if total > 0 else 0

print(f"   ✅ Successes: {successes}")
print(f"   ❌ Errors: {errors}")
print(f"   🎯 Success Rate: {success_rate:.1f}%")

if success_rate == 100:
    print("\n🎉 All requests completed successfully!")
elif success_rate >= 50:
    print("\n⚠️ Some requests failed - check the results above")
else:
    print("\n❌ Most requests failed - investigate connection issues")

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7592e25f6d50>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7592e26b9090>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7592e26bafd0>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7592e26bb250>


Connection ID: /subscriptions/0721e282-2773-4021-af16-e00641ed5e36/resourceGroups/rg-apim-basic-public/providers/Microsoft.CognitiveServices/accounts/ai-foundry-ijordjnftsf7w/projects/project-for-keyvault/connections/Azure-Rest-Github, Name: Azure-Rest-Github, Type: ConnectionType.REMOTE_TOOL Default: True
Connection ID: /subscriptions/0721e282-2773-4021-af16-e00641ed5e36/resourceGroups/rg-apim-basic-public/providers/Microsoft.CognitiveServices/accounts/ai-foundry-ijordjnftsf7w/projects/project-for-keyvault/connections/MS-learn, Name: MS-learn, Type: ConnectionType.REMOTE_TOOL Default: False
Gateway connections: {'model_gateway_connection_static': None, 'model_gateway_connection_dynamic': None, 'ai_gateway_connection_static': None, 'ai_gateway_connection_dynamic': None}
Using connection: None
No gateway connection found for project https://ai-foundry-ijordjnftsf7w.services.ai.azure.com/api/projects/project-for-keyvault, skipping...
Connection ID: /subscriptions/0721e282-2773-4021-af16-